In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from math import log2

factors = [1,1,1,1,1/2,1/4,1/8,1/16,1/32]

In [ ]:
class WSConv2d(nn.Module): #weighted-scale a.k.a. eq lr
  def __init__(self,in_channels,out_channels,kernel_size = 3, stride =1 , padding = 1, gain = 2):
    super().__init__()
    self.conv = nn.Conv2d(in_channels,out_channels,kernel_size,stride,padding)
    self.scale = (gain/(in_channels*kernel_size **2)) ** 0.5
    self.bias = self.conv.bias # what is it an why?
    self.conv.bias = None

    #init conv layer
    nn.init.normal_(self.conv.weight)#what is it an why?
    nn.init.zeros_(self.bias) #what is it an why?

  def forward(self,x):
    return self.conv(x*self.scale) + self.bias.view(1,self.bias.shape[0],1,1) #how it works?

In [ ]:
class PixelNorm(nn.Module):
  def __init__(self):
    super().__init__()
    self.epsilon = 1e-8

  def forward(self,x):
    return x / torch.sqrt(torch.mean(x**2,dim=1,keepdim=True) + self.epsilon)

In [ ]:
class ConvBlock(nn.Module):
  def __init__(self,in_channels,out_channels,use_pixelnorm = True):
    super().__init__()
    self.conv1 = WSConv2d(in_channels,out_channels)
    self.conv2 = WSConv2d(out_channels,out_channels)
    self.leaky = nn.LeakyReLU(0.2)
    self.pn = PixelNorm()
    self.use_pn = use_pixelnorm
  def forward(self,x):
    x = self.leaky(self.conv1(x))
    x = self.pn(x) if self.use_pn else x
    x = self.leaky(self.conv2(x))
    x = self.pn(x) if self.use_pn else x
    return x

In [ ]:
class Generator(nn.Module):
  def __init__ (self,z_dim,in_channels,img_channels = 3):
    super().__init__()
    self.initial = nn.Sequential(
        PixelNorm(),
        nn.ConvTranspose2d(z_dim,in_channels,4,1,0), #1x1 -> 4x4
        nn.LeakyReLU(0.2),
        WSConv2d(in_channels,in_channels),
        nn.LeakyReLU(0.2),
        PixelNorm(),
    )

    self.initial_rgb = WSConv2d(in_channels,img_channels,kernel_size = 1, stride = 1 , padding = 0)
    self.prog_blocks, self.rgb_layers = nn.ModuleList(), nn.ModuleList([self.initial_rgb])

    for i in range(len(factors) - 1):
      conv_in_channels = int(in_channels * factors[i])
      conv_out_channels = int(in_channels * factors[i+1])
      self.prog_blocks.append(ConvBlock(conv_in_channels,conv_out_channels))
      self.rgb_layers.append(WSConv2d(conv_out_channels,img_channels,kernel_size = 1, stride = 1,padding=0))

  def fade_in(self,alpha,upscaled,generated):
    return torch.tanh(alpha * generated + (1-alpha) * upscaled)

  def forward(self,x,alpha,steps): # steps = 0 (4x4), steps = 1(8x8) ...
    out = self.initial(x)

    if steps == 0:
      return self.initial_rgb(out)

    for step in range(steps):
      upscaled = F.interpolate(out,scale_factor = 2,mode = "nearest") # how it works
      out = self.prog_blocks[step](upscaled) # how it works

    final_upscaled = self.rgb_layers[steps - 1](upscaled)
    final_out = self.rgb_layers[steps](out)

    return self.fade_in(alpha,final_upscaled,final_out)

In [ ]:
class Discriminator(nn.Module):

  def __init__(self, noise_dim,in_channels,img_channels = 3):
    super().__init__()
    self.prog_blocks, self.rgb_layers = nn.ModuleList(), nn.ModuleList()
    self.leaky = nn.LeakyReLU(0.2)

    for i in range(len(factors)- 1, 0, -1):
      conv_in_channels = int(in_channels * factors[i])
      conv_out_channels = int(in_channels * factors[i-1])
      self.prog_blocks.append(ConvBlock(conv_in_channels,conv_out_channels,use_pixelnorm = False))
      self.rgb_layers.append(WSConv2d(img_channels,conv_in_channels,kernel_size = 1, stride = 1,padding = 0))

    self.initial_rgb = WSConv2d(img_channels,in_channels,kernel_size = 1, stride = 1,padding = 0)
    self.rgb_layers.append(self.initial_rgb)
    self.avg_pool = nn.AvgPool2d(kernel_size = 2, stride = 2)

    self.final_block = nn.Sequential(
        WSConv2d(in_channels + 1, in_channels, kernel_size = 3, stride = 1, padding =1),
        nn.LeakyReLU(0.2),
        WSConv2d(in_channels, in_channels, kernel_size = 4, stride = 1, padding =0),
        nn.LeakyReLU(0.2),
        WSConv2d(in_channels, 1, kernel_size = 1, stride = 1, padding =0),
    )

  def fade_in(self,alpha,downscaled,out):
    return alpha * out + (1-alpha) * downscaled

  def minibatch_std(self,x): # need to better understand
    batch_statistics = torch.std(x,dim=0).mean().repeat(x.shape[0],1,x.shape[2],x.shape[3])
    return torch.cat([x,batch_statistics],dim =1)

  def forward(self,x,alpha,steps): # steps = 0 (4x4), steps = 1(8x8) ...
    cur_step = len(self.prog_blocks) - steps
    out = self.leaky(self.rgb_layers[cur_step](x))

    if steps == 0:
      out = self.minibatch_std(out)
      return self.final_block(out).view(out.shape[0],-1)

    downscaled = self.leaky(self.rgb_layers[cur_step +1](self.avg_pool(x)))
    out = self.avg_pool(self.prog_blocks[cur_step](out))
    out = self.fade_in(alpha,downscaled,out)

    for step in range(cur_step+1,len(self.prog_blocks)):
      out = self.prog_blocks[step](out)
      out = self.avg_pool(out)

    out = self.minibatch_std(out)
    return self.final_block(out).view(out.shape[0],-1)

In [ ]:
###CONFIG###
import cv2

START_TRAIN_AT_IMG_SIZE = 128
DATASET = 'celeb_dataset'
CHECKPOINT_GEN = "generator.pth"
CHECKPOINT_CRITIC = "critic.pth"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
SAVE_MODEL = False
LOAD_MODEL = False
LEARNING_RATE = 1e-3
BATCH_SIZES = [32, 32, 32, 16, 16, 16, 16, 8, 4]
CHANNELS_IMG = 3
NOISE_DIM = 256  # should be 512 in original paper
IN_CHANNELS = 256  # should be 512 in original paper
CRITIC_ITERATIONS = 1
LAMBDA_GP = 10
PROGRESSIVE_EPOCHS = [30] * len(BATCH_SIZES)
FIXED_NOISE = torch.randn(8, NOISE_DIM, 1, 1).to(DEVICE)
NUM_WORKERS = 4

In [ ]:
###TEST###
gen = Generator(NOISE_DIM, IN_CHANNELS, img_channels=3)
critic = Discriminator(NOISE_DIM, IN_CHANNELS, img_channels=3)

for img_size in [4, 8, 16, 32, 64, 128, 256, 512, 1024]:
  num_steps = int(log2(img_size / 4))
  x = torch.randn((1,NOISE_DIM,1,1))
  z = gen(x, 0.5, steps=num_steps)
  assert z.shape == (1, 3, img_size, img_size)
  out = critic(z, alpha=0.5, steps=num_steps)
  assert out.shape == (1, 1)
  print(f"Success! At img size: {img_size}")

Success! At img size: 4
Success! At img size: 8
Success! At img size: 16
Success! At img size: 32
Success! At img size: 64
Success! At img size: 128
Success! At img size: 256
Success! At img size: 512
Success! At img size: 1024


In [ ]:
###UTILS###
import random
import numpy as np
import os
import torchvision
from torchvision.utils import save_image
from scipy.stats import truncnorm

# Print losses occasionally and print to tensorboard
def plot_to_tensorboard(writer, loss_critic, loss_gen, real, fake, tensorboard_step):
    writer.add_scalar("Loss Critic", loss_critic, global_step=tensorboard_step)

    with torch.no_grad():
        # take out (up to) 8 examples to plot
        img_grid_real = torchvision.utils.make_grid(real[:8], normalize=True)
        img_grid_fake = torchvision.utils.make_grid(fake[:8], normalize=True)
        writer.add_image("Real", img_grid_real, global_step=tensorboard_step)
        writer.add_image("Fake", img_grid_fake, global_step=tensorboard_step)


def gradient_penalty(critic, real, fake, alpha, train_step, device="cpu"):
    BATCH_SIZE, C, H, W = real.shape
    beta = torch.rand((BATCH_SIZE, 1, 1, 1)).repeat(1, C, H, W).to(device)
    interpolated_images = real * beta + fake.detach() * (1 - beta)
    interpolated_images.requires_grad_(True)

    # Calculate critic scores
    mixed_scores = critic(interpolated_images, alpha, train_step)

    # Take the gradient of the scores with respect to the images
    gradient = torch.autograd.grad(
        inputs=interpolated_images,
        outputs=mixed_scores,
        grad_outputs=torch.ones_like(mixed_scores),
        create_graph=True,
        retain_graph=True,
    )[0]
    gradient = gradient.view(gradient.shape[0], -1)
    gradient_norm = gradient.norm(2, dim=1)
    gradient_penalty = torch.mean((gradient_norm - 1) ** 2)
    return gradient_penalty


def save_checkpoint(model, optimizer, filename="my_checkpoint.pth.tar"):
    print("=> Saving checkpoint")
    checkpoint = {
        "state_dict": model.state_dict(),
        "optimizer": optimizer.state_dict(),
    }
    torch.save(checkpoint, filename)


def load_checkpoint(checkpoint_file, model, optimizer, lr):
    print("=> Loading checkpoint")
    checkpoint = torch.load(checkpoint_file, map_location="cuda")
    model.load_state_dict(checkpoint["state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer"])

    # If we don't do this then it will just have learning rate of old checkpoint
    # and it will lead to many hours of debugging \:
    for param_group in optimizer.param_groups:
        param_group["lr"] = lr

def generate_examples(gen, steps, truncation=0.7, n=100):
    """
    Tried using truncation trick here but not sure it actually helped anything, you can
    remove it if you like and just sample from torch.randn
    """
    gen.eval()
    alpha = 1.0
    for i in range(n):
        with torch.no_grad():
            noise = torch.tensor(truncnorm.rvs(-truncation, truncation, size=(1, NOISE_DIM, 1, 1)), device=DEVICE, dtype=torch.float32)
            img = gen(noise, alpha, steps)
            save_image(img*0.5+0.5, f"saved_examples/img_{i}.png")
    gen.train()

In [ ]:
###TRAIN###
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm

torch.backends.cudnn.benchmarks = True


def get_loader(image_size):
    transform = transforms.Compose(
        [
            transforms.Resize((image_size, image_size)),
            transforms.ToTensor(),
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.Normalize(   [0.5 for _ in range(CHANNELS_IMG)],    [0.5 for _ in range(CHANNELS_IMG)],     ),
        ]
    )
    batch_size = BATCH_SIZES[int(log2(image_size / 4))]
    dataset = datasets.ImageFolder(root=DATASET, transform=transform)
    loader = DataLoader(dataset,batch_size=batch_size,shuffle=True,num_workers=NUM_WORKERS,pin_memory=True,)
    return loader, dataset

def train_fn(critic,gen,loader,dataset,step,alpha,opt_critic,opt_gen,tensorboard_step,writer,scaler_gen,scaler_critic,):
  loop = tqdm(loader, leave = True)
  for batch_idx, (real,_) in enumerate(loop):
    real = real.to(DEVICE)
    cur_batch_size = real.shape[0]

    noise = torch.randn(cur_batch_size,NOISE_DIM,1,1).to(DEVICE)

    with torch.cuda.amp.autocast(): # need to remember what is autocast
      fake = gen(noise,alpha,step)
      critic_real = critic(real,alpha,step)
      critic_fake = critic(fake.detach(),alpha,step)
      gp = gradient_penalty(critic,real,fake,alpha,step,device = DEVICE)
      loss_critic =( -(torch.mean(critic_real) - torch.mean(critic_fake)) + LAMBDA_GP * gp + (0.001 * torch.mean(critic_real**2)))

    opt_critic.zero_grad()
    scaler_critic.scale(loss_critic).backward()
    scaler_critic.step(opt_critic)
    scaler_critic.update()

    with torch.cuda.amp.autocast(): # need to remember what is autocast
      gen_fake = critic(fake,alpha,step)
      loss_gen = -torch.mean(gen_fake)

    opt_gen.zero_grad()
    scaler_gen.scale(loss_gen).backward()
    scaler_gen.step(opt_gen)
    scaler_gen.update()

    alpha += cur_batch_size/(len(dataset) * PROGRESSIVE_EPOCHS[step] * 0.5)
    alpha = min(alpha,1)

    if batch_idx % 500 == 0:
      with torch.no_grad():
        fixed_fakes = gen(FIXED_NOISE, alpha, step) * 0.5 + 0.5
      plot_to_tensorboard(
        writer,
        loss_critic.item(),
        loss_gen.item(),
        real.detach(),
        fixed_fakes.detach(),
        tensorboard_step,
      )
      tensorboard_step += 1

      loop.set_postfix(
        gp=gp.item(),
        loss_critic=loss_critic.item(),
      )

    return tensorboard_step, alpha
def main():
  gen = Generator(NOISE_DIM,IN_CHANNELS,img_channels=CHANNELS_IMG).to(DEVICE)
  critic = Discriminator(NOISE_DIM, IN_CHANNELS, img_channels=CHANNELS_IMG).to(DEVICE)

  opt_gen = optim.Adam(gen.parameters(),lr = LEARNING_RATE,betas = (0.0,0.99))
  opt_critic = optim.Adam(critic.parameters(),lr = LEARNING_RATE,betas = (0.0,0.99))

  scaler_critic = torch.cuda.amp.GradScaler() #remember what is it
  scaler_gen = torch.cuda.amp.GradScaler()

  writer = SummaryWriter(f"logs/gan")

  if LOAD_MODEL:
    load_checkpoint(
        CHECKPOINT_GEN,gen,opt_gen,LEARNING_RATE,
    )
    load_checkpoint(
        CHECKPOINT_CRITIC,critic,opt_critic,LEARNING_RATE
    )

  gen.train()
  critic.train()
  tensorboard_step = 0
  step = int(log2(START_TRAIN_AT_IMG_SIZE/4))
  for num_epochs in PROGRESSIVE_EPOCHS[step:]:
        alpha = 1e-5
        loader, dataset = get_loader(4 * 2 ** step)  # 4->0, 8->1, 16->2, 32->3, 64 -> 4
        print(f"Current image size: {4 * 2 ** step}")

        for epoch in range(num_epochs):
            print(f"Epoch [{epoch+1}/{num_epochs}]")
            tensorboard_step, alpha = train_fn(critic,gen,loader,dataset,step,alpha,opt_critic,opt_gen,tensorboard_step,writer,scaler_gen,scaler_critic,)

            if SAVE_MODEL:
                save_checkpoint(gen, opt_gen, filename=CHECKPOINT_GEN)
                save_checkpoint(critic, opt_critic, filename=CHECKPOINT_CRITIC)

        step += 1  # progress to the next img size

In [ ]:
main()